In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
from layers import PosTagger
from metrics import MaskedLoss

In [2]:
 tf.config.run_functions_eagerly(False)

In [19]:
def prep_ds(file):
    inp = ['']
    targ = [[]]
    idx = 0
    for line in file:
        try:
            word, tag, _ = line.split(' ')
            inp[idx]+= word + ' '
            targ[idx].append(tag)
        except:
            targ[idx]=np.asarray(targ[idx])
            idx +=1
            inp.append('')
            targ.append([])
    return tf.convert_to_tensor(inp), targ


In [20]:
with open("Data/train.txt",mode='r', encoding = "utf-8") as f:
    X_train, y_train = prep_ds(f)
with open("Data/test.txt",mode='r', encoding = "utf-8") as f:
    X_test, y_test = prep_ds(f)

In [5]:
y_train = tf.ragged.constant(y_train[:-1])
X_train = X_train[:-1]
X_test = X_test[:-1]
y_test = tf.ragged.constant(y_test[:-1])

In [6]:
print(X_train[:2])
print(y_train[:2])

tf.Tensor(
[b"Confidence in the pound is widely expected to take another sharp dive if trade figures for September , due for release tomorrow , fail to show a substantial improvement from July and August 's near-record deficits . "
 b"Chancellor of the Exchequer Nigel Lawson 's restated commitment to a firm monetary policy has helped to prevent a freefall in sterling over the past week . "], shape=(2,), dtype=string)
<tf.RaggedTensor [[b'NN', b'IN', b'DT', b'NN', b'VBZ', b'RB', b'VBN', b'TO', b'VB', b'DT',
  b'JJ', b'NN', b'IN', b'NN', b'NNS', b'IN', b'NNP', b',', b'JJ', b'IN',
  b'NN', b'NN', b',', b'VB', b'TO', b'VB', b'DT', b'JJ', b'NN', b'IN',
  b'NNP', b'CC', b'NNP', b'POS', b'JJ', b'NNS', b'.']                    ,
 [b'NNP', b'IN', b'DT', b'NNP', b'NNP', b'NNP', b'POS', b'VBN', b'NN',
  b'TO', b'DT', b'NN', b'JJ', b'NN', b'VBZ', b'VBN', b'TO', b'VB', b'DT',
  b'NN', b'IN', b'NN', b'IN', b'DT', b'JJ', b'NN', b'.']                 ]>


In [7]:
preproc_url =  'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_url = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1'
bert_urls = {'preproc':preproc_url,'encoder':bert_url}

In [14]:
model = PosTagger(110,bert_urls, attention=None)

In [15]:
model.compile(loss=MaskedLoss(), optimizer = 'adam')

In [16]:
model.fit(x=X_train,y=y_train,batch_size=32, epochs=2)

Epoch 1/2
280/280 [==============================] - 29s 92ms/step - loss: 278.1182 - accuracy: 0.8292
Epoch 2/2
280/280 [==============================] - 25s 89ms/step - loss: 92.5120 - accuracy: 0.8881


In [17]:
model(X_train[:1])

<tf.RaggedTensor [[b'NN', b'IN', b'DT', b'NN', b'VBZ', b'RB', b'VBN', b'TO', b'VB', b'DT',
  b'JJ', b'NN', b'IN', b'NN', b'NNS', b'IN', b'NNP', b',', b'JJ', b'IN',
  b'NN', b'NN', b',', b'VB', b'TO', b'VB', b'DT', b'JJ', b'NN', b'IN',
  b'NNP', b'CC', b'NNP', b'POS', b'JJ', b'NN', b'.']]>

In [18]:
model.save('model dense')

INFO:tensorflow:Assets written to: model dense\assets


INFO:tensorflow:Assets written to: model dense\assets


In [ ]:
model = tf.keras.models.load_model('model dense' ,compile= False)

In [24]:
x = "i don't like it here"

In [25]:
def tag_finder(word,sentence):
    idx = []
    for i,j in enumerate(sentence.split(' ')):
        if j == word:
            idx.append(i)
    return [model([sentence])[0][n].numpy().decode('utf-8') for n in idx]

def accuracy(y_pred,y_true):
    if not isinstance(y_true,tf.RaggedTensor):
        y_true = tf.ragged.constant(y_true)

    matches = tf.math.equal(y_true,y_pred)
    matches = tf.cast(matches, tf.dtypes.int32)
    total_matches = tf.reduce_sum(matches).numpy()
    
    total = tf.size(y_true)
    return total_matches/total.numpy()

In [26]:
tag_finder("don't",x)

['VBP']

In [ ]:
accuracy(model.predict(X_test),y_test)